# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [7]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")

# Display sample data
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Luanda,AO,-8.8368,13.2343,78.80,83,40,8.05,Clouds
1,1,Castro,BR,-24.7911,-50.0119,61.84,90,100,10.02,Clouds
2,2,Dwarka,IN,22.2394,68.9678,77.23,47,2,10.60,Clear
3,3,Busselton,AU,-33.6500,115.3333,60.67,75,0,13.94,Clear
4,4,Klaksvik,FO,62.2266,-6.5890,49.12,90,3,12.21,Clear


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [8]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 70
What is the maximum temperature you would like for your vacation? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [9]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & ((city_data_df['Max Temp'] >= min_temp))]

# Display sample data
preferred_cities_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Luanda,AO,-8.8368,13.2343,78.80,83,40,8.05,Clouds
2,2,Dwarka,IN,22.2394,68.9678,77.23,47,2,10.60,Clear
5,5,Matale,LK,7.4721,80.6234,72.55,93,99,1.66,Clouds
13,13,Maxixe,MZ,-23.8597,35.3472,75.02,94,75,11.99,Clouds
14,14,San Pedro De Uraba,CO,8.2752,-76.3764,71.91,97,100,4.61,Clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [16]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Luanda,AO,-8.8368,13.2343,78.80,83,40,8.05,Clouds
2,2,Dwarka,IN,22.2394,68.9678,77.23,47,2,10.60,Clear
5,5,Matale,LK,7.4721,80.6234,72.55,93,99,1.66,Clouds
13,13,Maxixe,MZ,-23.8597,35.3472,75.02,94,75,11.99,Clouds
14,14,San Pedro De Uraba,CO,8.2752,-76.3764,71.91,97,100,4.61,Clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [17]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng
0,Luanda,AO,78.80,Clouds,-8.8368,13.2343
2,Dwarka,IN,77.23,Clear,22.2394,68.9678
5,Matale,LK,72.55,Clouds,7.4721,80.6234
13,Maxixe,MZ,75.02,Clouds,-23.8597,35.3472
14,San Pedro De Uraba,CO,71.91,Clouds,8.2752,-76.3764


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [21]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Luanda,AO,78.80,Clouds,-8.8368,13.2343,
2,Dwarka,IN,77.23,Clear,22.2394,68.9678,
5,Matale,LK,72.55,Clouds,7.4721,80.6234,
13,Maxixe,MZ,75.02,Clouds,-23.8597,35.3472,
14,San Pedro De Uraba,CO,71.91,Clouds,8.2752,-76.3764,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [23]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [24]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address['features'][0]['properties']['name']
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Luanda - nearest hotel: Hotel Forum
Dwarka - nearest hotel: The Dwarika Hotel
Matale - nearest hotel: The Grand Mountain Hotel
Maxixe - nearest hotel: Hotel Herry
San Pedro De Uraba - nearest hotel: No hotel found
Rikitea - nearest hotel: Chez Bianca & Benoit
Sao Filipe - nearest hotel: Bela Vista
Ponta Do Sol - nearest hotel: Estalagem Ponta do Sol
Puerto Ayora - nearest hotel: Hostal La Mirada De Solitario George
Yulara - nearest hotel: Outback Pioneer Hotel & Lodge
Amuntai - nearest hotel: Grand Balqis Hotel
Hilo - nearest hotel: Dolphin Bay Hotel
Cabo San Lucas - nearest hotel: Comfort Rooms
Labuhan - nearest hotel: No hotel found
Hermanus - nearest hotel: Aloe guest house
Mar Del Plata - nearest hotel: Nuevo Ostende
El Alto - nearest hotel: No hotel found
Gouyave - nearest hotel: No hotel found
Alibag - nearest hotel: Hira Laxmi Residency
Balabac - nearest hotel: No hotel found
Katherine - nearest hotel: No hotel found
Itarema - nearest hotel: Pousada Oásis
A

Bulnes - nearest hotel: Hotel Casa de Adobe
Cristalina - nearest hotel: Goyás
Road Town - nearest hotel: No hotel found
Pangai - nearest hotel: Billy's Place
Kutum - nearest hotel: No hotel found
Lakeside - nearest hotel: No hotel found
Kiunga - nearest hotel: No hotel found
Kasongo-Lunda - nearest hotel: No hotel found
Melo - nearest hotel: Hotel Crown
Vero Beach - nearest hotel: Hampton Inn & Suites
Baracoa - nearest hotel: Casa
Richards Bay - nearest hotel: Protea Hotel
Mogadishu - nearest hotel: SYL Hotel
Pak Phanang - nearest hotel: Pak Phanang Hotel
Ankazoabo - nearest hotel: No hotel found
Kingston - nearest hotel: Aphrodite Hotel
Lorengau - nearest hotel: Seeadler Bay Hotel
Doha - nearest hotel: Al MIrqab
Areia Branca - nearest hotel: Areia Branca Apart Hotel
Grand Gaube - nearest hotel: Coco Villa
Kawalu - nearest hotel: Hotel Surya
San Cristobal - nearest hotel: Terra Sur
Taoudenni - nearest hotel: No hotel found
Naravarikuppam - nearest hotel: No hotel found
Bocas Del Toro -

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Luanda,AO,78.80,Clouds,-8.8368,13.2343,Hotel Forum
2,Dwarka,IN,77.23,Clear,22.2394,68.9678,The Dwarika Hotel
5,Matale,LK,72.55,Clouds,7.4721,80.6234,The Grand Mountain Hotel
13,Maxixe,MZ,75.02,Clouds,-23.8597,35.3472,Hotel Herry
14,San Pedro De Uraba,CO,71.91,Clouds,8.2752,-76.3764,No hotel found
...,...,...,...,...,...,...,...
673,Zambezi,ZM,73.02,Rain,-13.5432,23.1047,No hotel found
675,Tignere,CM,85.28,Clouds,7.3667,12.6500,No hotel found
682,Puerto Escondido,MX,79.63,Clouds,15.8500,-97.0667,Hotel Barlovento
685,Aracati,BR,87.93,Clouds,-4.5617,-37.7697,Hotel Complexo Mirante das Gamboas


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [29]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != "No hotel found"]

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Luanda,AO,78.80,Clouds,-8.8368,13.2343,Hotel Forum
2,Dwarka,IN,77.23,Clear,22.2394,68.9678,The Dwarika Hotel
5,Matale,LK,72.55,Clouds,7.4721,80.6234,The Grand Mountain Hotel
13,Maxixe,MZ,75.02,Clouds,-23.8597,35.3472,Hotel Herry
16,Rikitea,PF,73.17,Clear,-23.1203,-134.9692,Chez Bianca & Benoit
17,Sao Filipe,CV,77.95,Clouds,14.8961,-24.4956,Bela Vista
20,Ponta Do Sol,PT,71.01,Clouds,32.6667,-17.1000,Estalagem Ponta do Sol
21,Puerto Ayora,EC,73.36,Clouds,-0.7393,-90.3518,Hostal La Mirada De Solitario George
23,Yulara,AU,74.43,Clouds,-25.2406,130.9889,Outback Pioneer Hotel & Lodge
30,Amuntai,ID,75.22,Rain,-2.4177,115.2494,Grand Balqis Hotel


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [30]:
# Create the output File (CSV)
output_data_file = "./WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label = "Hotel_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [43]:
# Configure the map plot
map_plot_hotel = clean_hotel_df.hvplot.points(
    "Lng", "Lat", geo=True, tiles="OSM", frame_width=700, frame_height=500,
    size = "Max Temp", scale = 0.5, color = 'City',
    hover_cols = ["City", "Hotel Name", "Country", "Current Description"]
    )

# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)